In [ ]:
# Obtain the Morgan descriptors for the chemicals in the
# PFAS database

In [230]:
import pandas as pd
import numpy as np
from rdkit import DataStructs
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [231]:
df = pd.read_csv('df_mordred_PFAS_MasterList.csv')

C:\Users\Adi\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (150,151,152,159,160,161,168,169,170,177,178,179,186,187,188,195,196,197,204,205,206,213,214,215,222,223,224,231,232,233,240,241,242,245,246,247,248,249,250,251,252,253,353,354,355,356,357,358,366,367,368,375,376,377,384,385,386,393,394,395,402,403,404,411,412,413,420,421,422,429,430,431,438,439,440,447,448,449,456,457,458,461,462,463,464,465,472,473,474,480,481,482,488,489,490,496,497,498,504,505,506,512,513,514,520,521,522,528,529,530,536,537,538,544,545,546,552,553,554,557,558,559,560,561,568,569,570,576,577,578,584,585,586,592,593,594,600,601,602,608,609,610,616,617,618,624,625,626,632,633,634,640,641,642,648,649,650,653,654,827,828,882,883,898,899,1120,1139,1199,1218,1278,1279,1280,1281,1282,1283,1284,1285,1286,1287,1288,1289,1290,1291,1294,1297,1299,1589) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, res

In [232]:
df.head()

,DTXSID,PREFERRED_NAME_x,INCHIKEY_x,SMILES_x,MOLECULAR_FORMULA_x,AVERAGE_MASS_x,C:M Ratio Master,C:M Ratio Lipid Normalized,Log C:M Ratio Lipid Normalized,C:M Ratio Non-Lipid Normalized,...,SRW10,TSRW10,MW,AMW,WPath,WPol,Zagreb1,Zagreb2,mZagreb1,mZagreb2
0,DTXSID80844621,"2,2,3-Trifluoro-3-(pentafluoroethoxy)propanoic...",XEZRGPDPHQWFHV-UHFFFAOYSA-N,OC(=O)C(F)(F)C(F)OC(F)(F)C(F)(F)F,C5H2F8O3,262.055,NaN,NaN,NaN,NaN,...,9.852247,48.801716,261.987620,14.554868,424,26,80,91,10.659722,3.270833
1,DTXSID00844623,"Fluoro[1,1,2,2-tetrafluoro-2-(trifluoromethoxy...",GHVHGLNEHMMXAV-UHFFFAOYSA-N,OC(=O)C(F)OC(F)(F)C(F)(F)OC(F)(F)F,C5H2F8O4,278.054,NaN,NaN,NaN,NaN,...,9.803280,49.832196,277.982534,14.630660,512,24,84,92,10.909722,3.465278
2,DTXSID60844624,Fluoro(heptafluoropropoxy)acetic acid,SOPKUPKNOTUECV-UHFFFAOYSA-N,OC(=O)C(F)OC(F)(F)C(F)(F)C(F)(F)F,C5H2F8O3,262.055,NaN,NaN,NaN,NaN,...,9.933095,48.927773,261.987620,14.554868,415,27,80,92,10.659722,3.277778
3,DTXSID70845743,Trifluoroacetic acid--8-fluoro-6-[(piperidin-3...,HIRJPQZSAAXQMS-UHFFFAOYSA-N,OC(=O)C(F)(F)F.FC1=C2C=NC=CC2=CC(OC2CCCNC2)=C1,C16H16F4N2O3,360.309,NaN,NaN,NaN,NaN,...,9.763708,51.101624,246.116841,7.458086,620,26,94,109,4.555556,3.972222
4,DTXSID30845744,Trifluoroacetic acid--8-fluoro-6-[(piperidin-4...,QSVWXQWXIQVXRP-UHFFFAOYSA-N,OC(=O)C(F)(F)F.FC1=C2C=NC=CC2=CC(OC2CCNCC2)=C1,C16H16F4N2O3,360.309,NaN,NaN,NaN,NaN,...,9.763708,51.101624,246.116841,7.458086,620,26,94,109,4.555556,3.972222


In [233]:
df.shape

(7986, 1838)

In [234]:
smiles = list(df['QSAR_READY_SMILES_x']) # extract the smiles column from the dataframe
smiles 

['OC(=O)C(F)(F)C(F)OC(F)(F)C(F)(F)F',
 'OC(=O)C(F)OC(F)(F)C(F)(F)OC(F)(F)F',
 'OC(=O)C(F)OC(F)(F)C(F)(F)C(F)(F)F',
 'FC1=C2C=NC=CC2=CC(OC2CCCNC2)=C1',
 'FC1=C2C=NC=CC2=CC(OC2CCNCC2)=C1',
 'FC1(F)C(F)(F)C(F)(C2=CC=C(Br)C=C2)C1(F)C1=CC=C(Br)C=C1',
 'FC1(F)C(F)(F)C2(F)C(F)(F)C1(F)C(F)(F)C(F)(F)C2(F)F',
 'FC(F)(F)C(=C=C[N+]#N)C(F)(F)F',
 'OC(=O)C(F)(C(F)C(F)(F)F)C(F)(F)F',
 'OC(=O)C1(F)C(F)(F)C1(F)F',
 'OC(=O)C1(C(F)(F)F)C(F)(F)C1(F)F',
 'OC(=O)C(F)C(F)(C(F)(F)F)C(F)(F)F',
 'OC(=O)C(F)(C(F)F)C(F)F',
 'NC1=C(Br)C=C(OC(F)(F)C(F)C(F)(F)F)C=C1Br',
 'NC1=C(Br)N=C(OC(C(F)(F)F)C(F)(F)F)C=C1',
 '[O-][N+](=O)C1=CN=C(OC(C(F)(F)F)C(F)(F)F)C=C1',
 'NC1=C(Br)C(SC(F)(F)C(F)(F)C(F)(F)F)=C(Br)C=C1Br',
 'NC1=C(Br)C=C(C2=C1C=CC=N2)C(F)(C(F)(F)F)C(F)(F)F',
 'FC(F)(F)C(F)(F)CNC(=O)C1=CC(NC(=O)C2=CC=CC=C2)=CC=C1',
 'FC(F)(F)C(F)(F)C(F)(F)CNC(=O)C1=CC(NC(=O)C2=CC=CC=C2)=CC=C1',
 'NCCCCCC=C(C(F)(F)C(F)(F)F)C(F)(F)C(F)(F)F',
 'CC(C=C(F)C(F)(F)F)C(F)(F)F',
 'COCC=C(F)C(F)(F)C(F)(F)C(F)(F)F',
 'CN(C)CC=C(F)C(F)(F)C

In [235]:
frags = {}
count = 0

for i in smiles:
    mol = Chem.MolFromSmiles(i) # convert the SMILES to mol format 
    fp1 = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=1024) # use the mol file to calculate Morgan fingerprint bits
    # show 10 of the set bits:
    list(fp1.GetOnBits())

    fp_arr = np.zeros((1,)) # initialize empty numpy array
    fp_arr
    DataStructs.ConvertToNumpyArray(fp1, fp_arr) # add the morgan bits to the empty numpy array
    inchi = df.iloc[count,2] # extract the INCHIKEY
    frags[inchi] = fp_arr # create a dict with the INCHI as key and the morgan bits array as value
    
    
    count += 1

In [236]:
frags

{'XEZRGPDPHQWFHV-UHFFFAOYSA-N': array([0., 1., 0., ..., 0., 0., 0.]),
 'GHVHGLNEHMMXAV-UHFFFAOYSA-N': array([0., 1., 0., ..., 0., 0., 0.]),
 'SOPKUPKNOTUECV-UHFFFAOYSA-N': array([0., 1., 0., ..., 0., 0., 0.]),
 'HIRJPQZSAAXQMS-UHFFFAOYSA-N': array([0., 0., 0., ..., 0., 0., 0.]),
 'QSVWXQWXIQVXRP-UHFFFAOYSA-N': array([0., 0., 0., ..., 0., 0., 0.]),
 'PTHWBGWGLNJPEZ-UHFFFAOYSA-N': array([0., 0., 0., ..., 0., 0., 0.]),
 'OYNPVYZYLUYCDR-UHFFFAOYSA-N': array([0., 0., 0., ..., 0., 0., 0.]),
 'UCXKYBLLGOYNCZ-UHFFFAOYSA-N': array([0., 0., 0., ..., 0., 0., 0.]),
 'LBLWHJJKMUMQEE-UHFFFAOYSA-N': array([0., 1., 0., ..., 0., 0., 0.]),
 'HXUVAKMCUAKGOU-UHFFFAOYSA-N': array([0., 0., 0., ..., 0., 0., 0.]),
 'AEKBSACOYMDZQM-UHFFFAOYSA-N': array([0., 0., 0., ..., 0., 0., 0.]),
 'JLRPXCDIADORQL-UHFFFAOYSA-N': array([0., 1., 0., ..., 0., 0., 0.]),
 'PBPPVCSRXYDNTQ-UHFFFAOYSA-N': array([0., 1., 0., ..., 0., 0., 0.]),
 'OJAXIOZVWGLZNM-UHFFFAOYSA-N': array([0., 1., 0., ..., 0., 0., 0.]),
 'NCWWZHNTXZMGBE-UHF

In [237]:
df_frag = pd.DataFrame.from_dict(frags, orient='index') # convert dict to df
df_frag = df_frag.reset_index()
df_frag.rename(columns = {'index':'INCHIKEY_x'}, inplace = True)
df_frag # each column is a morgan bit 

,INCHIKEY_x,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,XEZRGPDPHQWFHV-UHFFFAOYSA-N,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,GHVHGLNEHMMXAV-UHFFFAOYSA-N,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,SOPKUPKNOTUECV-UHFFFAOYSA-N,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HIRJPQZSAAXQMS-UHFFFAOYSA-N,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,QSVWXQWXIQVXRP-UHFFFAOYSA-N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7977,UQXUUCPMHUZPSW-UHFFFAOYSA-N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7978,JMRCQLAMQGMJDF-UHFFFAOYSA-N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7979,JSECXTYNFBONSF-UHFFFAOYNA-N,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7980,DOZBMDQURDJHIU-UHFFFAOYSA-N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [238]:
df_merged = df.merge(df_frag, on = 'INCHIKEY_x') # merge MC_DB with morgan bits dataframe
df_merged

,DTXSID,PREFERRED_NAME_x,INCHIKEY_x,SMILES_x,MOLECULAR_FORMULA_x,AVERAGE_MASS_x,C:M Ratio Master,C:M Ratio Lipid Normalized,Log C:M Ratio Lipid Normalized,C:M Ratio Non-Lipid Normalized,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,DTXSID80844621,"2,2,3-Trifluoro-3-(pentafluoroethoxy)propanoic...",XEZRGPDPHQWFHV-UHFFFAOYSA-N,OC(=O)C(F)(F)C(F)OC(F)(F)C(F)(F)F,C5H2F8O3,262.055,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,DTXSID00844623,"Fluoro[1,1,2,2-tetrafluoro-2-(trifluoromethoxy...",GHVHGLNEHMMXAV-UHFFFAOYSA-N,OC(=O)C(F)OC(F)(F)C(F)(F)OC(F)(F)F,C5H2F8O4,278.054,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,DTXSID60844624,Fluoro(heptafluoropropoxy)acetic acid,SOPKUPKNOTUECV-UHFFFAOYSA-N,OC(=O)C(F)OC(F)(F)C(F)(F)C(F)(F)F,C5H2F8O3,262.055,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,DTXSID70845743,Trifluoroacetic acid--8-fluoro-6-[(piperidin-3...,HIRJPQZSAAXQMS-UHFFFAOYSA-N,OC(=O)C(F)(F)F.FC1=C2C=NC=CC2=CC(OC2CCCNC2)=C1,C16H16F4N2O3,360.309,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,DTXSID30845744,Trifluoroacetic acid--8-fluoro-6-[(piperidin-4...,QSVWXQWXIQVXRP-UHFFFAOYSA-N,OC(=O)C(F)(F)F.FC1=C2C=NC=CC2=CC(OC2CCNCC2)=C1,C16H16F4N2O3,360.309,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7981,DTXSID501019231,9:3Fluorotelomer carboxylate,UQXUUCPMHUZPSW-UHFFFAOYSA-N,CCOC(=O)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F)(F)C(F...,C12H5F19O2,542.140,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7982,DTXSID201019244,"1,3-Dichloro-1,1,2,2-tetrafluoropropane",JMRCQLAMQGMJDF-UHFFFAOYSA-N,FC(F)(Cl)C(F)(F)CCl,C3H2Cl2F4,184.940,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7983,DTXSID201019268,"3-Chloro-1,1,1,2-tetrafluoropropane",JSECXTYNFBONSF-UHFFFAOYNA-N,FC(CCl)C(F)(F)F,C3H3ClF4,150.500,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7984,DTXSID401019638,"1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,8-Heptadecaflu...",DOZBMDQURDJHIU-UHFFFAOYSA-N,CN(CCOCCOCCOCCOCCOCCOCCOCCOCCO)S(=O)(=O)C(F)(F...,C27H40F17NO11S,909.650,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [239]:
df_merged.to_csv('Fingerprints_PFAS_DB.csv')